## Import Packages

In [70]:
%pip install --upgrade panel jupyter_bokeh holoviews -q

Note: you may need to restart the kernel to use updated packages.


In [71]:
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import cartopy.crs as ccrs
import panel as pn

pn.extension()

In [72]:
def check_and_reproject(df):
    """
    Checks the coordinate reference system (CRS) of the input DataFrame and reprojects it to EPSG:4326 if necessary.

    Parameters:
    df (GeoDataFrame): The input DataFrame containing spatial data.

    Returns:
    GeoDataFrame: The reprojected DataFrame.
    """
    if df.crs != 'EPSG:4326':
        df = df.to_crs('EPSG:4326')
    else:
        df=df
    return df

In [73]:
def process_paddocks(land_parcels, user_paddocks):
    """
    Args:
        land_parcels (geopandas.GeoDataFrame): GeoDataFrame representing the land parcels.
        user_paddocks (geopandas.GeoDataFrame): GeoDataFrame representing the user-defined paddocks.

    Returns:
        tuple: A tuple containing the following:
            - land_parcels_unify (geopandas.GeoDataFrame): GeoDataFrame representing the dissolved land parcels.
            - land_parcels_centroid (geopandas.GeoSeries): GeoSeries representing the centroid of the land parcels.
            - inside_areas (geopandas.GeoDataFrame): GeoDataFrame representing the areas of the user-input boundaries that fall inside the land parcels.
            - outside_areas (geopandas.GeoDataFrame): GeoDataFrame representing the areas areas of the user-input boundaries that outside the land parcels.
    """
    land_parcels_unify = land_parcels.dissolve()
    land_parcels_centroid = land_parcels_unify.centroid

    difference_areas = gpd.overlay(user_paddocks, land_parcels, how='symmetric_difference') 
    outside_areas = gpd.overlay(difference_areas, land_parcels, how='difference')
    inside_areas = user_paddocks.clip(land_parcels)
    
    return land_parcels_unify, land_parcels_centroid, inside_areas, outside_areas


In [74]:
def export_geojson(inside_areas, outside_areas):
    """
    Export two files:
     - the modified user-input boundaries
     - the areas that fall outside the land parcels
     
    The files will be projected to EPSG:4326 before being exported.
    """
    output_crs = '4326'

    inside_areas.to_file('output-data/modified-user-boundaries.geojson', driver='GeoJSON', crs=output_crs)
    outside_areas.to_file('output-data/outside-parcel-user-boundaries.geojson', driver='GeoJSON', crs=output_crs)


In [75]:
# TODO: add widgets to let user drop in their onw files
cpes = gpd.read_file('/workspace/notebooks/sandbox/data/input-data/CPES/cpes.shp')
paddocks = gpd.read_file('/workspace/notebooks/sandbox/data/input-data/Paddock boundaries/paddock-boundaries.shp').to_crs(cpes.crs)

# set up map tiles for basemap
map_tiles = hv.element.tiles.EsriImagery().opts(width=600, height=400)

cpes_reproject=check_and_reproject(cpes)
paddocks_reproject=check_and_reproject(paddocks)

In [76]:
land_parcels_unify, land_parcels_centroid, inside_areas, outside_areas = process_paddocks(cpes_reproject, paddocks_reproject)

/tmp/ipykernel_19083/1636014326.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  land_parcels_centroid = land_parcels_unify.centroid


In [77]:
land_parcels_unify.head()

,geometry,oid_1,property_i,property_n,property_t,enterprise,property_a,lga,st_area_sh,st_perimet,object_id
0,"MULTIPOLYGON (((114.99811 -28.05150, 114.99812...",1873890,1000530.0,KALINYA,Rural,Agricultural,3359.753668,NORTHAMPTON,4.329696e+07,50050.137382,141


In [78]:
outside_areas.head()

,id,oid_1,property_i,property_n,property_t,enterprise,property_a,lga,st_area_sh,st_perimet,object_id,geometry
0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((114.93998 -28.00452, 114.93828 -28.0..."
1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((114.88951 -28.00797, 114.88951..."


In [79]:
inside_areas.head()

,id,geometry
2,None,"MULTIPOLYGON (((114.93780 -28.00488, 114.93064..."
1,None,"POLYGON ((114.94180 -27.99394, 114.94066 -27.9..."
0,None,"POLYGON ((114.92415 -27.98180, 114.92589 -27.9..."


In [87]:
parcels_map = land_parcels_unify.hvplot(alpha=0.5, color='slategrey', geo=True, width=1800, height=1800)
outside_map = outside_areas.hvplot(geo=True, line_color = 'red', alpha=0, line_alpha=1, line_width=2, width=1800, height=1800)
inside_map = inside_areas.hvplot(geo=True, line_color = 'lime', alpha=0, line_alpha=1, line_width=2, width=1800, height=1800)

In [88]:
combined_map = map_tiles * parcels_map * outside_map * inside_map

In [89]:
pn.panel(combined_map).show()

Launching server at http://localhost:33287


In [83]:
#export_geojson(inside_areas, outside_areas)